<a href="https://colab.research.google.com/github/Elwing-Chou/ml0930/blob/master/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/additional/train.zip")
f.extractall()

In [ ]:
import pandas as pd
import glob
dogs = glob.glob("train/dog.*")
cats = glob.glob("train/cat.*")
df = pd.DataFrame({
    "path":dogs + cats,
    "target":[0] * len(dogs) + [1] * len(cats)
})
df